<a href="https://colab.research.google.com/github/Sarvesh1814/US-Book-Recommendation-System/blob/main/Item_Item_Collaborative_Assigment3_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mounting Drive

In [2]:
# Mounting Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Importing Important Libraries

In [3]:
# Importing Important Libraries...
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
# Loading Data...
ratings = pd.read_csv('/content/drive/MyDrive/Semester-2/Recommendation System/Assignment 3/ratings.csv')
books = pd.read_csv('/content/drive/MyDrive/Semester-2/Recommendation System/Assignment 3/Books.csv')

<ipython-input-4-b4e23db902fc>:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('/content/drive/MyDrive/Semester-2/Recommendation System/Assignment 3/Books.csv')


# Visualizing Data

In [5]:
# Visualizing ratings data...
ratings.head()

,Unnamed: 0,User-ID,ISBN,Book-Rating
0,0,276725,034545104X,0
1,1,276726,0155061224,5
2,8,276744,038550120X,7
3,16,276747,0060517794,9
4,17,276747,0451192001,0


In [6]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [7]:
# Checking for duplicates
print(ratings.duplicated().sum())
print(books.duplicated().sum())

0
0


In [8]:
# Printing the shape of dataframes
print(ratings.shape)
print(books.shape)

(768068, 4)
(271360, 8)


# Merging Data

In [9]:
# Merging the ratings and books dataframe on ISBN
ratings_with_name = ratings.merge(books, on='ISBN')
print(ratings_with_name.shape)

(746466, 11)


In [10]:
ratings_with_name.head()

,Unnamed: 0,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
1,15433,2313,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
2,27533,6543,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
3,36088,8680,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
4,40794,10314,034545104X,9,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...


# Filtering Based on Users

Reasoning:-

* There are lots of users who have rated very less number of books and similar for the books.
* So, we have decided to take the genuine user and famous books only.

In [11]:
# Filtering merged dataframe based on the users...
X = ratings_with_name.groupby('User-ID').count()['Book-Rating'] > 100
genuine_users = X[X].index
genuine_users

Int64Index([   254,    507,    882,   1435,   1733,   2033,   2110,   2276,
              2358,   2766,
            ...
            274808, 275020, 275970, 276463, 276680, 277427, 277639, 278137,
            278188, 278418],
           dtype='int64', name='User-ID', length=1166)

In [12]:
# Filtered ratings...
filtered_ratings = ratings_with_name[ratings_with_name['User-ID'].isin(genuine_users)]
filtered_ratings

,Unnamed: 0,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
2,27533,6543,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
4,40794,10314,034545104X,9,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
5,101553,23768,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
7,123655,28523,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
13,325432,77480,034545104X,8,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
...,...,...,...,...,...,...,...,...,...,...,...
746440,1149615,276680,0812881311,0,Streets of Gold,Evan Hunter,1985,Madison Books,http://images.amazon.com/images/P/0812881311.0...,http://images.amazon.com/images/P/0812881311.0...,http://images.amazon.com/images/P/0812881311.0...
746441,1149618,276680,0865055041,0,Kitchen (Historic Communities (Paperback)),Bobbie Kalman,1990,Crabtree Publishing Company,http://images.amazon.com/images/P/0865055041.0...,http://images.amazon.com/images/P/0865055041.0...,http://images.amazon.com/images/P/0865055041.0...
746442,1149619,276680,086505505X,0,Home Crafts (Historic Communities (Paperback)),Bobbie Kalman,1990,Crabtree Publishing Company,http://images.amazon.com/images/P/086505505X.0...,http://images.amazon.com/images/P/086505505X.0...,http://images.amazon.com/images/P/086505505X.0...
746443,1149625,276680,1564407284,0,"The Old Sturbridge Village Cookbook, 2nd : Aut...",Caroline Sloat,1995,Globe Pequot,http://images.amazon.com/images/P/1564407284.0...,http://images.amazon.com/images/P/1564407284.0...,http://images.amazon.com/images/P/1564407284.0...


# Filtering Based on Recipes

In [13]:
# Filtering Based on recipes...
y = filtered_ratings.groupby('Book-Title').count()['Book-Rating']>=50
famous_books = y[y].index
famous_books

Index(['1984', '1st to Die: A Novel', '2nd Chance', '4 Blondes',
       'A Beautiful Mind: The Life of Mathematical Genius and Nobel Laureate John Nash',
       'A Bend in the Road', 'A Case of Need',
       'A Child Called \It\": One Child's Courage to Survive"',
       'A Civil Action', 'A Cry In The Night',
       ...
       'Winter Solstice', 'Wish You Well', 'Without Remorse',
       'Wizard and Glass (The Dark Tower, Book 4)', 'Wuthering Heights',
       'Year of Wonders', 'You Belong To Me',
       'Zen and the Art of Motorcycle Maintenance: An Inquiry into Values',
       'Zoya', '\O\" Is for Outlaw"'],
      dtype='object', name='Book-Title', length=764)

In [14]:
# Final ratings dataframe...
final_ratings = filtered_ratings[filtered_ratings['Book-Title'].isin(famous_books)]

In [15]:
# Dropping Duplicates if any...
final_ratings.drop_duplicates()

,Unnamed: 0,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
58,5966,278418,038550120X,0,A Painted House,JOHN GRISHAM,2001,Doubleday,http://images.amazon.com/images/P/038550120X.0...,http://images.amazon.com/images/P/038550120X.0...,http://images.amazon.com/images/P/038550120X.0...
59,25981,6251,038550120X,0,A Painted House,JOHN GRISHAM,2001,Doubleday,http://images.amazon.com/images/P/038550120X.0...,http://images.amazon.com/images/P/038550120X.0...,http://images.amazon.com/images/P/038550120X.0...
63,64980,13273,038550120X,0,A Painted House,JOHN GRISHAM,2001,Doubleday,http://images.amazon.com/images/P/038550120X.0...,http://images.amazon.com/images/P/038550120X.0...,http://images.amazon.com/images/P/038550120X.0...
64,65999,13552,038550120X,0,A Painted House,JOHN GRISHAM,2001,Doubleday,http://images.amazon.com/images/P/038550120X.0...,http://images.amazon.com/images/P/038550120X.0...,http://images.amazon.com/images/P/038550120X.0...
65,74604,16106,038550120X,0,A Painted House,JOHN GRISHAM,2001,Doubleday,http://images.amazon.com/images/P/038550120X.0...,http://images.amazon.com/images/P/038550120X.0...,http://images.amazon.com/images/P/038550120X.0...
...,...,...,...,...,...,...,...,...,...,...,...
744640,1130643,271284,0440910927,0,The Rainmaker,John Grisham,1995,Island,http://images.amazon.com/images/P/0440910927.0...,http://images.amazon.com/images/P/0440910927.0...,http://images.amazon.com/images/P/0440910927.0...
744963,1133070,271705,B0001PIOX4,0,Fahrenheit 451,Ray Bradbury,1993,Simon &amp; Schuster,http://images.amazon.com/images/P/B0001PIOX4.0...,http://images.amazon.com/images/P/B0001PIOX4.0...,http://images.amazon.com/images/P/B0001PIOX4.0...
745724,1142740,274549,0451518012,0,Animal Farm,George Orwell,1956,Signet Book,http://images.amazon.com/images/P/0451518012.0...,http://images.amazon.com/images/P/0451518012.0...,http://images.amazon.com/images/P/0451518012.0...
745786,1143468,274808,0449701913,0,Homecoming,Cynthia Voigt,1983,Fawcett Books,http://images.amazon.com/images/P/0449701913.0...,http://images.amazon.com/images/P/0449701913.0...,http://images.amazon.com/images/P/0449701913.0...


# Train Test Split

In [16]:
import numpy as np

# Group the data frame by the 'UserID' column
grouped = final_ratings.groupby('User-ID')

# Calculate the number of reviews for each user
review_counts = grouped['ISBN'].count()

# Create an empty list to store the test data frames
test_data = []

# Iterate through each group of reviews for a user
for user, group in grouped:
    # Determining the number of reviews to include in the test data frame
    test_size = int(np.ceil(0.2 * len(group)))
    # Split the group of reviews into train and test data frames
    train = group.iloc[:-test_size]
    test = group.iloc[-test_size:]
    # Append the test data frame to the list of test data frames
    test_data.append(test)

# Concatenate all of the test data frames into a single test data frame
test_data = pd.concat(test_data)

# Drop the groups of reviews from the original data frame that were split into the test data frame
train_data = final_ratings[~final_ratings.index.isin(test_data.index)]

In [17]:
test_data.head()

,Unnamed: 0,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
143726,10230,254,0446356832,0,The Sands of Time,Sidney Sheldon,1989,Warner Books,http://images.amazon.com/images/P/0446356832.0...,http://images.amazon.com/images/P/0446356832.0...,http://images.amazon.com/images/P/0446356832.0...
143888,10241,254,0451137965,0,Thinner,Stephen King,1985,New Amer Library,http://images.amazon.com/images/P/0451137965.0...,http://images.amazon.com/images/P/0451137965.0...,http://images.amazon.com/images/P/0451137965.0...
143961,10242,254,0451150244,0,Pet Sematary,Stephen King,1984,Signet Book,http://images.amazon.com/images/P/0451150244.0...,http://images.amazon.com/images/P/0451150244.0...,http://images.amazon.com/images/P/0451150244.0...
143971,10243,254,0451167317,8,The Dark Half,Stephen King,1994,Signet Book,http://images.amazon.com/images/P/0451167317.0...,http://images.amazon.com/images/P/0451167317.0...,http://images.amazon.com/images/P/0451167317.0...
144211,10253,254,0451526341,8,Animal Farm,George Orwell,2004,Signet,http://images.amazon.com/images/P/0451526341.0...,http://images.amazon.com/images/P/0451526341.0...,http://images.amazon.com/images/P/0451526341.0...


In [18]:
train_data.head()

,Unnamed: 0,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
58,5966,278418,038550120X,0,A Painted House,JOHN GRISHAM,2001,Doubleday,http://images.amazon.com/images/P/038550120X.0...,http://images.amazon.com/images/P/038550120X.0...,http://images.amazon.com/images/P/038550120X.0...
59,25981,6251,038550120X,0,A Painted House,JOHN GRISHAM,2001,Doubleday,http://images.amazon.com/images/P/038550120X.0...,http://images.amazon.com/images/P/038550120X.0...,http://images.amazon.com/images/P/038550120X.0...
63,64980,13273,038550120X,0,A Painted House,JOHN GRISHAM,2001,Doubleday,http://images.amazon.com/images/P/038550120X.0...,http://images.amazon.com/images/P/038550120X.0...,http://images.amazon.com/images/P/038550120X.0...
64,65999,13552,038550120X,0,A Painted House,JOHN GRISHAM,2001,Doubleday,http://images.amazon.com/images/P/038550120X.0...,http://images.amazon.com/images/P/038550120X.0...,http://images.amazon.com/images/P/038550120X.0...
65,74604,16106,038550120X,0,A Painted House,JOHN GRISHAM,2001,Doubleday,http://images.amazon.com/images/P/038550120X.0...,http://images.amazon.com/images/P/038550120X.0...,http://images.amazon.com/images/P/038550120X.0...


In [19]:
print("Shape of Train Data:- ",train_data.shape)
print("Shape of Test Data:- ",test_data.shape)

Shape of Train Data:-  (51595, 11)
Shape of Test Data:-  (13485, 11)


In [22]:
train_data.to_csv('/content/drive/MyDrive/Semester-2/Recommendation System/train.csv')
test_data.to_csv('test.csv')

In [25]:
final_ratings.to_csv('/content/drive/MyDrive/Semester-2/Recommendation System/ratings_final.csv')

# Creating Pivot Table

In [25]:
# create pivot table for train data
train_pt = train_data.pivot_table(index='ISBN', columns='User-ID', values='Book-Rating')
# calculate similarity scores
similarity_scores = cosine_similarity(train_pt.fillna(0))
# create pivot table for test data
test_pt = test_data.pivot_table(index='ISBN', columns='User-ID', values='Book-Rating')
# test_pt.fillna(0,inplace=True)
# train_pt.fillna(0,inplace=True)

In [27]:
train_pt

User-ID,254,507,882,1435,1733,2033,2110,2276,2358,2766,...,274808,275020,275970,276463,276680,277427,277639,278137,278188,278418
ISBN,,,,,,,,,,,,,,,,,,,,,
002542730X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN
002542730x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0060008024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0060008296,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
006001203X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1573225789,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
157322930X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1573229326,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
test_pt

User-ID,254,507,882,1435,1733,2033,2110,2276,2358,2766,...,274808,275020,275970,276463,276680,277427,277639,278137,278188,278418
ISBN,,,,,,,,,,,,,,,,,,,,,
0020697406,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
002542730x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0060008032,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
006001203X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0060086246,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
B00007CWQC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B00009EF82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B0000T6KHI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Recommendation Engine

In [29]:
def get_top_k_recommendations(user_id, k, rated_items=[]):
    """
    Get top k recommendations for a given user ID.
    Returns:
    List of top k recommended recipe names for the user.
    """
    if user_id not in train_pt.columns:
        return []

    # update user's ratings with rated items from previous rounds
    for book, rating in rated_items:
        if book in train_pt.index:
            train_pt[user_id][book] = rating

    # Fetching the rated books from train pivot table for given user_id 
    rated_books = train_pt[user_id]
    rated_books = rated_books[rated_books.notna()]
    # Finding the unrated books for given user id
    unrated_books = train_pt.index.difference(rated_books.index)
    predicted_ratings = []

    '''
    Predicting Rating for each book in unrated books.
    '''
    for book in unrated_books:
        similarity_scores_for_book = similarity_scores[train_pt.index.get_loc(book)]
        user_ratings = train_pt[user_id].fillna(0)
        numerator = (user_ratings * similarity_scores_for_book).sum()
        denominator = similarity_scores_for_book.sum()

        if denominator == 0:
            predicted_rating = 0
        else:
            predicted_rating = numerator / denominator

        predicted_ratings.append((book, predicted_rating))
        
    top_book_names = [book for book, predicted_rating in sorted(predicted_ratings, key=lambda x: x[1], reverse=True)[:k]]
    return top_book_names

In [38]:
def main():
  """
    Runs the main recommendation loop for the recipe recommender system.

    The loop prompts the user to enter their ID and displays the top k recommended
    recipes for the user based on their ratings and similarity to other users. The user
    can then select a recipe and provide a rating for it, and the loop will continue
    until the user is satisfied or there are no more recommendations left.

    """
   
  # initialize list of rated items
  rated_items = []

  # Taking user_id from the user
  user_id = int(input("Enter user id:"))

  # loop until user is satisfied or there are no more recommendations
  while True:
      # get top k recommendations for user
      recommendations = get_top_k_recommendations(user_id, 5, rated_items)

      # check if there are any recommendations left
      if len(recommendations) == 0:
          print("No more recommendations")
          break

      # display recommendations to user
      print("Here are your recommendations:")
      for i, recommendation in enumerate(recommendations):
          print(f"{i+1}"+". " f"{final_ratings[final_ratings['ISBN'] == recommendation]['Book-Title'].values[0]}")
      
      # ask user to select item they like or if they want more recommendations
      selection = input("Enter number of item you like or 'm' for more recommendations: ")

      # if user wants more recommendations, continue to next round
      if selection == 'm':
          continue

      # get the index of the selected item
      try:
          selection_idx = int(selection) - 1
          selected_item = recommendations[selection_idx]
      except:
          print("Invalid selection")
          continue

      # add selected item and rating to list of rated items
      rating = float(input(f"What rating would you give {selected_item} (0-5)? "))
      rated_items.append((selected_item, rating))

      # if user is satisfied, show ingredients of selected item
      if input("Are you satisfied? (y/n) ") == 'y':
          print(f"Book Name for {selected_item}:")
          print(final_ratings[final_ratings['ISBN'] == selected_item]['Book-Title'].values[0])
          break

if __name__ == '__main__':
    main()

Enter user id:2033
Here are your recommendations:
1. Dragonfly in Amber
2. Neverwhere
3. Catering to Nobody
4. Angels
5. The Golden Compass (His Dark Materials, Book 1)
Enter number of item you like or 'm' for more recommendations: 3
What rating would you give 0312042779 (0-5)? 4
Are you satisfied? (y/n) y
Book Name for 0312042779:
Catering to Nobody
